In [18]:
!pip install transformers
!pip install sentencepiece
!pip install torch


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [19]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("Helsinki-NLP/opus-mt-en-de")
model = AutoModelForSeq2SeqLM.from_pretrained("Helsinki-NLP/opus-mt-en-de")


/usr/local/lib/python3.9/dist-packages/transformers/models/marian/tokenization_marian.py:194: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


In [20]:
import nltk
nltk.download('gutenberg')
nltk.download('punkt')
from nltk.corpus import gutenberg as gut

print(gut.fileids())

['austen-emma.txt', 'austen-persuasion.txt', 'austen-sense.txt', 'bible-kjv.txt', 'blake-poems.txt', 'bryant-stories.txt', 'burgess-busterbrown.txt', 'carroll-alice.txt', 'chesterton-ball.txt', 'chesterton-brown.txt', 'chesterton-thursday.txt', 'edgeworth-parents.txt', 'melville-moby_dick.txt', 'milton-paradise.txt', 'shakespeare-caesar.txt', 'shakespeare-hamlet.txt', 'shakespeare-macbeth.txt', 'whitman-leaves.txt']


[nltk_data] Downloading package gutenberg to /root/nltk_data...
[nltk_data]   Package gutenberg is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [40]:
macbeth_text = nltk.corpus.gutenberg.raw('shakespeare-macbeth.txt')
macbeth_text[:500]

"[The Tragedie of Macbeth by William Shakespeare 1603]\n\n\nActus Primus. Scoena Prima.\n\nThunder and Lightning. Enter three Witches.\n\n  1. When shall we three meet againe?\nIn Thunder, Lightning, or in Raine?\n  2. When the Hurley-burley's done,\nWhen the Battaile's lost, and wonne\n\n   3. That will be ere the set of Sunne\n\n   1. Where the place?\n  2. Vpon the Heath\n\n   3. There to meet with Macbeth\n\n   1. I come, Gray-Malkin\n\n   All. Padock calls anon: faire is foule, and foule is faire,\nHouer through "

In [23]:
import numpy as np
from keras.models import Sequential, load_model
from keras.utils import to_categorical
from random import randint
import re

def preprocess_text(sen):
    # Remove punctuations and numbers
    sentence = re.sub('[^a-zA-Z]', ' ', sen)

    # Single character removal
    sentence = re.sub(r"\s+[a-zA-Z]\s+", ' ', sentence)

    # Removing multiple spaces
    sentence = re.sub(r'\s+', ' ', sentence)

    return sentence.lower()

macbeth_text = preprocess_text(macbeth_text)
text=macbeth_text[:450]

In [25]:
#input_text = "This is a sample input text. It contains multiple sentences. We want to summarize this text using BERT."
sentences = text.split(".")
summary = ". ".join(sentences[:2]) + "."


In [26]:
from transformers import BertTokenizer, BertLMHeadModel

model = BertLMHeadModel.from_pretrained('bert-base-uncased')
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')


If you want to use `BertLMHeadModel` as a standalone, add `is_decoder=True.`
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertLMHeadModel: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertLMHeadModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertLMHeadModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [34]:
input_ids = tokenizer.encode(text, return_tensors="pt")
#summary_ids = model.generate(input_ids, num_beams=4, max_length=150, early_stopping=True)
summary_ids = model.generate(input_ids, 
                              num_beams=4, 
                              max_length=150, 
                              temperature=1.0, 
                              do_sample=True,
                              early_stopping=True,
                              no_repeat_ngram_size=2,
                              length_penalty=1.0,
                              num_return_sequences=1)


summary_text = tokenizer.decode(summary_ids[0], skip_special_tokens=True)


In [37]:
print(input_ids)

tensor([[  101,  1996, 19817, 18655,  2666,  1997, 25182,  2011,  2520,  8101,
          2552,  2271, 26927,  7606,  8040,  8913,  2532, 21111,  8505,  1998,
          7407,  4607,  2093, 12566,  2043,  4618,  2057,  2093,  3113,  2153,
          2063,  1999,  8505,  7407,  2030,  1999,  4542,  2063,  2043,  1996,
         25124, 20934, 12866,  2589,  2043,  1996,  7151, 14162,  2063,  2439,
          1998,  2180,  2638,  2008,  2097,  2022,  9413,  2063,  1996,  2275,
          1997,  3103,  2638,  2073,  1996,  2173, 21210,  2239,  1996,  9895,
          2045,  2000,  3113,  2007, 25182,  2272,  3897, 15451,  4939,  2035,
         11687,  7432,  4455,  2019,  2239,  4189,  2063,  2003, 12487,  2063,
          1998, 12487,  2063,  2003,  4189,  2063,  7570, 13094,  2083,  1996,
          9666,  3351,  1998, 10882, 24658,  2666,  1037, 16363,  4654,   102]])


In [38]:
print(summary_ids)

tensor([[  101,  1996, 19817, 18655,  2666,  1997, 25182,  2011,  2520,  8101,
          2552,  2271, 26927,  7606,  8040,  8913,  2532, 21111,  8505,  1998,
          7407,  4607,  2093, 12566,  2043,  4618,  2057,  2093,  3113,  2153,
          2063,  1999,  8505,  7407,  2030,  1999,  4542,  2063,  2043,  1996,
         25124, 20934, 12866,  2589,  2043,  1996,  7151, 14162,  2063,  2439,
          1998,  2180,  2638,  2008,  2097,  2022,  9413,  2063,  1996,  2275,
          1997,  3103,  2638,  2073,  1996,  2173, 21210,  2239,  1996,  9895,
          2045,  2000,  3113,  2007, 25182,  2272,  3897, 15451,  4939,  2035,
         11687,  7432,  4455,  2019,  2239,  4189,  2063,  2003, 12487,  2063,
          1998, 12487,  2063,  2003,  4189,  2063,  7570, 13094,  2083,  1996,
          9666,  3351,  1998, 10882, 24658,  2666,  1037, 16363,  4654,   102,
          1012,  1012,  1025,  1025,  1064,   999,   999,  1064,  1064,  1031,
          1031,  1064,  1033,  1033,  1064,  1027,  

In [35]:
print(summary_text)

the tragedie of macbeth by william shakespeare actus primus scoena prima thunder and lightning enter three witches when shall we three meet againe in thunder lightning or in raine when the hurley burley done when the battaile lost and wonne that will be ere the set of sunne where the place vpon the heath there to meet with macbeth come gray malkin all padock calls anon faire is foule and foule is faire houer through the fogge and filthie ayre ex.. ; ; |!! | | [ [ | ] ] | = = : : = ( ( ).! ) )?? = + + ( : ),, ( o o


In [36]:
!pip install rouge

from rouge import Rouge

rouge = Rouge()

# Generate the summary
#input_ids = tokenizer.encode(text, return_tensors="pt")
#summary_ids = model.generate(input_ids, num_beams=4, max_length=100, early_stopping=True)
#summary_text = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

# Calculate ROUGE scores
scores = rouge.get_scores(summary_text, text)
print(scores)


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
[{'rouge-1': {'r': 1.0, 'p': 0.8108108108108109, 'f': 0.8955223831142793}, 'rouge-2': {'r': 1.0, 'p': 0.7181818181818181, 'f': 0.8359788311133507}, 'rouge-l': {'r': 1.0, 'p': 0.8108108108108109, 'f': 0.8955223831142793}}]
